In [1]:
suppressMessages(library(dplyr))
suppressMessages(library('glmnet'))
suppressMessages(library(data.table))
suppressMessages(library('plink2R'))

In [2]:
MODELS_DIR <- '/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR'
TMP_DIR <- '/cellar/users/jlz014/fusion_twas-master/tmp/TCGA_BRCA_EUR_imputed_females_filtered'

In [3]:
models_files <- Sys.glob(file.path(MODELS_DIR,'*.RDat'))
tmp_bedfiles <- Sys.glob(file.path(TMP_DIR,'*tmp.bed'))
plink_base <- c(lapply(tmp_bedfiles, function(x) substr(x, 1, nchar(x)-4)))

In [4]:
# LUT for paths to weights and plink files for test set
files_lut <- fread('tcga-brca_enet_models_files_test_patients.txt',col.names=c('ensg','weights','plink'))

# Parse file to read in to bash script

In [5]:
significant_models <- read.table('tcga-brca_enet_models_significant_qval_with_gene_names.txt',header=T)

In [6]:
# subset LUT to genes with significant models
sub_lut <- filter(files_lut, ensg %in% significant_models$ensg)
head(sub_lut)

ensg,weights,plink
ENSG00000149084,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/EUR.ENSG00000149084.wgt.RDat,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.ENSG00000149084
ENSG00000128039,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/EUR.ENSG00000128039.wgt.RDat,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.ENSG00000128039
ENSG00000065308,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/EUR.ENSG00000065308.wgt.RDat,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.ENSG00000065308
ENSG00000184743,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/EUR.ENSG00000184743.wgt.RDat,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.ENSG00000184743
ENSG00000143256,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/EUR.ENSG00000143256.wgt.RDat,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.ENSG00000143256
ENSG00000114544,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/EUR.ENSG00000114544.wgt.RDat,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.ENSG00000114544


In [9]:
tail(sub_lut)

,ensg,weights,plink
1450,ENSG00000214617,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/EUR.ENSG00000214617.wgt.RDat,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.ENSG00000214617
1451,ENSG00000134061,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/EUR.ENSG00000134061.wgt.RDat,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.ENSG00000134061
1452,ENSG00000180155,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/EUR.ENSG00000180155.wgt.RDat,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.ENSG00000180155
1453,ENSG00000092094,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/EUR.ENSG00000092094.wgt.RDat,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.ENSG00000092094
1454,ENSG00000144596,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/EUR.ENSG00000144596.wgt.RDat,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.ENSG00000144596
1455,ENSG00000215124,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/EUR.ENSG00000215124.wgt.RDat,/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.ENSG00000215124


In [8]:
write.table(sub_lut, file='tcga-brca_enet_models_files_test_patients_significant_models.txt',quote=F, row.names=F, sep='\t')

# Test prediction

In [ ]:
# test on ENSG00000204310
test_gene <- 'ENSG00000204310'
files <- files_lut[which(files_lut$ensg==test_gene),]
files

In [ ]:
load(files$weights,verbose=T)
head(wgt.matrix)

In [ ]:
# load expression data for test set
test_expr <- fread('/data/cellardata/users/jlz014/TCGA/expression/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt')

In [ ]:
# get list of all patients in test set
test_patients <- colnames(test_expr)[-c(1:4)]

In [ ]:
# print row corresponding to test gene
test_row <- test_expr[which(test_expr$gene==test_gene),]
test_row

In [ ]:
# read plink files for test_gene
genos <- read_plink('/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/1_10/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.ENSG00000204310', impute='avg')
rownames(genos$bed) <- sapply(rownames(genos$bed), function(x) unlist(strsplit(x,':'))[1])

In [ ]:
test_out <- t(t(genos$bed) * wgt.matrix[, 'enet'])

In [ ]:
write.table(head(as.data.frame(rowSums(test_out))), file='test.txt', sep='\t', col.names=F, quote=F)